# Test Model-Scoring Service

This notebook uses newly created data to test the model-scoring service - i.e. it uses data generated for the period `t+1` to test a model trained with all data up until period `t`.

## Imports

In [1]:
import re
from urllib.request import urlopen
from typing import Dict, Tuple

import requests
import boto3 as aws
import numpy as np
import pandas as pd

## Download Newly Generated Dataset

In [2]:
latest_data_url = ('http://bodywork-ml-ops-project.s3.eu-west-2.amazonaws.com'
            '/datasets/regression-dataset-2021-01-12.csv')

latest_data = pd.read_csv(urlopen(latest_data_url))
latest_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1440 entries, 0 to 1439
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   y       1440 non-null   float64
 1   X       1440 non-null   float64
dtypes: float64(2)
memory usage: 22.6 KB


## Score Latest Data using Current Scoring-Service API Endpoint

We use the model-scoring REST API endpoint to get predictions for every instance in the new dataset. We use the known labels together with the scores to compute errors.

In [3]:
def get_model_score(url: str, features: Dict[str, float]) -> float:
    """Request score from REST API for a single instance of data."""
    session = requests.Session()
    session.mount(url, requests.adapters.HTTPAdapter(max_retries=3))
    response = session.post(url, json=features)
    return response.json()['prediction']


def analyse_model_score(score: float, label: float) -> Tuple[float, float, float]:
    """Compute performance metrics for model score."""
    absolute_percentage_error = abs(score / label - 1)
    return (score, label, absolute_percentage_error)


def generate_model_test_results(url: str, test_data: pd.DataFrame) -> pd.DataFrame:
    """Get test results for all test data."""
    def single_test_result(X: float, label: float) -> Tuple[float, float, float]:
        score = get_model_score(url, {'X': X})
        test_result = analyse_model_score(score, label)
        return test_result
    
    test_data = [single_test_result(row.X, row.y) for row in test_data.itertuples()]
    return pd.DataFrame(test_data, columns=['score', 'label', 'APE'])

        
scoring_service_url = 'http://localhost:5000/score/v1'
test_results = generate_model_test_results(scoring_service_url, latest_data)

# Analyse Test Results

Computing test metrics using scores and labels.

In [4]:
def test_metrics(test_results: pd.DataFrame) -> pd.DataFrame:
    MAPE = test_results.APE.mean()
    R2 = test_results.score.corr(test_results.label)
    MR = test_results.APE.max()
    return pd.DataFrame({'MAPE': [MAPE], 'R2': R2, 'MR': [MR]})


metrics = test_metrics(test_results)
for k, v in metrics.to_dict().items():
    print(f'{k}: {v[0]:.2f}')

MAPE: 0.18
R2: 0.82
MR: 16.97


## Persist Test Results

Upload test metrics to AWS S3.

In [5]:
def make_filename(data_url: str) -> str:
    """Generate model and metrics filenames from data URL."""
    data_date = re.findall('20[0-9][0-9]-[0-1][0-9]-[0-3][0-9]', data_url)[0]
    metrics_filename = f'regressor-{data_date}.csv'
    return metrics_filename
    

metrics_filename = make_filename(latest_data_url)
metrics.to_csv(metrics_filename, header=True, index=False)
    
s3_client = aws.client('s3')
s3_client.upload_file(metrics_filename,
    'bodywork-ml-ops-project',
    f'test-metrics/{metrics_filename}'
)
print(f'uploaded {metrics_filename} to s3://bodywork-ml-ops-project/test-metrics')

uploaded regressor-2021-01-12.csv to s3://bodywork-ml-ops-project/test-metrics
